In [29]:
from sentiment import *

In [30]:
test_queries = [
    "Sorry? That's all you've got? A simple \"sorry\" won't fix the mess of a stay I had. What are you going to do about it?",
    "Oh, you \"hear\" me? That's just great. Listening is one thing, but I want action, not just words! What are you going to do about it?.",
    "Well, you better make it quick! And it better be the best room you have, or there will be even more complaints coming your way.",
    "The room that I booked at your hotel was not what was advertised. It was dirty and had a musty smell. I am very disappointed and will not be staying here again.",
    "I appreciate your apology. Could you please let me know what steps can be taken to address this issue?",
    "Thank you for offering to switch me to a clean room. Can you ensure that the new room will be in a better condition than the first one?",
    "That sounds great, thank you. Could you also let me know how long it will take to prepare the new room?",
    "I understand it may take some time, but spending the entire day waiting for a new room is quite inconvenient. Is there any way to expedite this process, or perhaps offer some form of compensation for the inconvenience caused?",
    "That would be appreciated. What form of compensation are you considering?"
]

In [33]:
for query in test_queries:
    print(query)
    print("NLTK:", analyze_sentiment_nltk(query))
    print("TextBlob:", analyze_sentiment_textblob(query))
    print("Transformers:", analyze_sentiment_transformer(test_queries[0]))
    print()

Sorry? That's all you've got? A simple "sorry" won't fix the mess of a stay I had. What are you going to do about it?
NLTK: Slightly Positive
TextBlob: Slightly Negative
Transformers: Very Negative

Oh, you "hear" me? That's just great. Listening is one thing, but I want action, not just words! What are you going to do about it?.
NLTK: Positive
TextBlob: Positive
Transformers: Very Negative

Well, you better make it quick! And it better be the best room you have, or there will be even more complaints coming your way.
NLTK: Very Positive
TextBlob: Positive
Transformers: Very Negative

The room that I booked at your hotel was not what was advertised. It was dirty and had a musty smell. I am very disappointed and will not be staying here again.
NLTK: Very Negative
TextBlob: Very Negative
Transformers: Very Negative

I appreciate your apology. Could you please let me know what steps can be taken to address this issue?
NLTK: Positive
TextBlob: Neutral
Transformers: Very Negative

Thank you 

# Agent Testing

## Updates:
- Customer agent now uses GPT-4o to generate responses
- Historical context for support agents now summarizes the chat history

In [1]:
from agents import *

sender_agent = mAgentCustomer()

In [69]:
def get_history_first():
    chat_history = [AIMessage(content="                AmericanAir I have been trying to get a refund for my canceled flight for over a month now. Every time I call, I am told that it will be processed soon but it has been weeks and I still haven't received anything. This is unacceptable and frustrating. ")]
    return chat_history

def get_history_second():
    chat_history = [AIMessage(content="                AmericanAir I have been trying to get a refund for my canceled flight for over a month now. Every time I call, I am told that it will be processed soon but it has been weeks and I still haven't received anything. This is unacceptable and frustrating. "),
                    HumanMessage(content="sorry for your bad experience, could you please provide your order ID?")]
    return chat_history

prompt = "sorry for your bad experience, could you please provide your order ID?"

In [7]:
result = sender_agent.invoke({"input": prompt, "chat_history": get_history_first(), "civil": 0})
response = result

**Sample Response (POST-UPDATE)** : "Are you serious? You should have all my information lready. It's 12345678. Do your job and fix this mess."

In [35]:
historic_context = sender_agent.get_historical_context_chain().invoke({'question':prompt, "chat_history": get_history_first()})

**Sample Historic Context**: 'Could you please provide your order ID for the refund request regarding your canceled flight?'

### Historic Context

In [70]:
historic_context = info_agent.get_historical_info_context_chain().invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})

In [71]:
historic_context

'My order ID is 12345678. Please process my refund immediately.'

**Sample Historic Context:**
'My order ID is 12345678. Please resolve this issue promptly.'

Output does not always reflect historic context, but rather only the most recent information provided by the customer. The invocation used here already leverages GPT-4o. Therefore, we need to adjust the prompts to reflect the actual context.

In [109]:
client_completion = lcai.AzureOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_version="2024-05-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name="PROPILOT",
    model_name="gpt-4o",
)

def get_historical_context_chain():
    contextualize_q_system_prompt = """
    Your role is to ensure that {message} can be understood without the chat history.\
    
    The chat history contains an online conversation between a customer and a support representative.\
    The {message} is {sender}'s latest response in the chat.\
    
    Summarize the chat history in a way that provides context for the {message}.\
    """
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{sender}:{message}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()
    return contextualize_q_chain


In [107]:
historic_context_chat = get_historical_context_chain().invoke({'sender':"representative",'message':prompt, "chat_history": get_history_first()})
print(historic_context_chat)

The client expressed frustration over not receiving a refund for a canceled flight despite repeatedly contacting customer service over the past month. The representative responded by apologizing for the bad experience and asking for the client's order ID to assist further.


In [108]:
historic_context_info = get_historical_context_chain().invoke({'sender':"client",'message':response, "chat_history": get_history_second()})
print(historic_context_info)

The client has been trying to get a refund for a canceled flight for over a month. Despite multiple calls and assurances that the refund would be processed soon, they have not received it yet. Frustrated with the situation, the client expressed their dissatisfaction. The representative then asked for the order ID to assist further, to which the client responded angrily, stating that the company should already have their information and provided the order ID, 12345678, while demanding immediate resolution.


### Info Support Agent

In [121]:
info_agent = mAgentInfo()

response_cw_info = info_agent.invoke({'domain': "airlines",'complaint':response, "chat_history": get_history_second()})

**Sample Response:** 
['Apologize for the inconvenience caused.',
 'Confirm receipt of the order ID provided.',
 'Assure investigation and resolution.']
 
The response above is using the old historical context approach and GPT-4. Some of the cues are more about ettiquette and less about the information elements.

In [118]:
def agent_coworker_info():
    prompt = """Your role is to help a service representative by providing INFORMATIONAL SUPPORT. \
                The representative is chatting online with a customer complaining about {domain}.  \
                
                Given the chat history,
                provide 2-3 hints to help the representative's response.\
                The hints should direct the representative to do ONLY ONE of the following:\
                
                1) Inquire more details about the problem. OR \
                2) Request the customer to troubleshoot. OR \
                3) Provide a solution to resolve the customer's need. \
                
                Each cue should be a single phrase of less than 10 words.\
                Do NOT number the cues.\
                """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{domain}: {message}"),
        ]
    )

    chain = (RunnablePassthrough.assign(
        context=get_historical_context_chain()
    )
             | template
             | llmchat
             )

    def extract_cues(chain_output):
        cues_text = chain_output.content
        # Assuming each cue is separated by a newline in the chain_output.
        cues = cues_text.split('\n')
        # Filter out any empty strings or whitespace-only strings
        cues = [cue.strip() for cue in cues if cue.strip()]
        # Return the first 2 - 3 cues
        processed_cues = [re.sub(r'^\d+\.\s*', '', cue) for cue in cues]

        return processed_cues[:3]

    chain = chain | extract_cues

    return chain

In [123]:
response_support_info = agent_coworker_info().invoke({'domain': "airlines",'message':response, "chat_history": get_history_second(), "sender": "client"})
print(response_support_info)

['Ask if the payment method has changed.', 'Confirm the refund request was processed.', 'Check if additional information is required.']


**Sample Response (POST-UPDATE):**
['Confirm flight cancellation date.',
 'Ask if any email confirmation was received.',
 'Clarify preferred refund method.']